# Extracting `PoolCreated` events from Uniswap

This example shows how `evm_decode` and `evm_topic` can be used to find events and decode them. 

In [17]:
from nozzle.client import Client
from nozzle.util import to_hex
import os.path

# client_url = os.getenv("NOZZLE_URL", "grpc://127.0.0.1:1602")
client_url = "grpc+tcp://34.122.177.97:80"
client = Client(client_url)
print(client)


In [18]:
# The address of the Uniswap factory contract
uniswap_factory = "1F98431c8aD98523631AE4a59f267346ea31F984"

factory_path = "abis/factory.json"
pool_path = "abis/pool.json"

df = client.get_sql("select * from logs limit 1", read_all=True).to_pandas()


In [19]:
df

,block_hash,block_num,timestamp,tx_index,tx_hash,address,topic0,topic1,topic2,topic3,data,block_index,index,call_index,ordinal
0,b':\x1f\xbaZ\xbd\x9dAEyD\xe9\x1e\xd0\x97\xe09\...,52029,2015-08-08 06:22:55+00:00,0,b'!\x8bc-\x93#qG\x8d\x1a\xe5\xa0\x16 \xeb\xab\...,b'Ud\x88l\xa2\xc5\x18\xd1\x96N_\xce\xa4\xf4#\x...,b'\xa6i~\x97Nj2\x0fEC\x90\xbe\x03\xf7IU\xe8\x9...,b'MyFirstCoin\x00\x00\x00\x00\x00\x00\x00\x00\...,None,None,b'',0,0,1,8


In [20]:
client.get_sql("select count(*) logs_count from logs", read_all=True)

pyarrow.Table
logs_count: int64 not null
----
logs_count: [[3686440789]]

In [21]:
client.get_sql("select count(*) blocks_count, min(block_num) first_block, max(block_num) latest_block from blocks", read_all=True)

pyarrow.Table
blocks_count: int64
first_block: uint64
latest_block: uint64
----
blocks_count: [[19820001]]
first_block: [[0]]
latest_block: [[19820000]]

In [22]:
import time

def elapsed(start):
    return round(time.time() - start, 4)

# Processes a SQL query using the provided client and prints the timings.
# The first received batched is printed out, the rest only report the row count.
def process_query(client, query):
    print(query)
    start = time.time()

    result_stream = client.get_sql(query)
    print('time to establish stream: ', elapsed(start), 's')

    total_events = 0
    try:
        batch = next(result_stream)
        print('time for first batch ', elapsed(start), 's')
        total_events += batch.num_rows
        print(batch.to_pandas().map(to_hex))

        batch_start = time.time()

        for batch in result_stream:
            total_events += batch.num_rows
            print('received batch of ', batch.num_rows, ' rows in ', elapsed(batch_start), 's')
            batch_start = time.time()

        print('total rows: ', total_events)
        print('total time to consume the stream: ', elapsed(start), 's')
        print('rows/s: ', total_events / elapsed(start))

    except StopIteration:
        print("No more batches available in the result stream.")

In [23]:
from nozzle.util import Abi
import time

# Load a JSON ABI and get the 'PoolCreated' event
factory = Abi(factory_path)

pool_created_sig = factory.events["PoolCreated"].signature()

pool_created_query = f"""
    select pc.block_num,
       pc.dec['pool'] as pool,
       pc.dec['token0'] as token0,
       pc.dec['token1'] as token1,
       pc.dec['fee'] as fee,
       pc.dec['tickSpacing'] as tick_spacing
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{pool_created_sig}') as dec
              from logs l
             where l.address = arrow_cast(x'{uniswap_factory}', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('{pool_created_sig}')) pc
"""

# Explain analyze
# print(client.get_sql("explain analyze " + query, read_all=True).to_pandas()['plan'][0])

In [24]:
print(client.get_sql("explain analyze " + pool_created_query, read_all=True).to_pandas()['plan'][0])

ProjectionExec: expr=[block_num@0 as block_num, get_field(dec@1, pool) as pool, get_field(dec@1, token0) as token0, get_field(dec@1, token1) as token1, get_field(dec@1, fee) as fee, get_field(dec@1, tickSpacing) as tick_spacing], metrics=[output_rows=22259, elapsed_compute=954.9µs]
  ProjectionExec: expr=[block_num@0 as block_num, evm_decode(topic1@3, topic2@4, topic3@5, data@6, PoolCreated(address indexed token0,address indexed token1,uint24 indexed fee,int24 tickSpacing,address pool)) as dec], metrics=[output_rows=22259, elapsed_compute=11.880099ms]
    CoalesceBatchesExec: target_batch_size=8192, metrics=[output_rows=22259, elapsed_compute=867.993896ms]
      FilterExec: address@1 = 31,152,67,28,138,217,133,35,99,26,228,165,159,38,115,70,234,49,249,132 AND topic0@2 = 120,60,202,28,4,18,221,13,105,94,120,69,104,201,109,162,233,194,47,249,137,53,122,46,139,29,155,43,78,107,113,24, metrics=[output_rows=22259, elapsed_compute=50.555775404s]
        ParquetExec: file_groups={30 groups: [

In [25]:
process_query(client, pool_created_query)


    select pc.block_num,
       pc.dec['pool'] as pool,
       pc.dec['token0'] as token0,
       pc.dec['token1'] as token1,
       pc.dec['fee'] as fee,
       pc.dec['tickSpacing'] as tick_spacing
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'PoolCreated(address indexed token0,address indexed token1,uint24 indexed fee,int24 tickSpacing,address pool)') as dec
              from logs l
             where l.address = arrow_cast(x'1F98431c8aD98523631AE4a59f267346ea31F984', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('PoolCreated(address indexed token0,address indexed token1,uint24 indexed fee,int24 tickSpacing,address pool)')) pc

time to establish stream:  0.1103 s
time for first batch  110.3418 s
     block_num                                        pool  \
0     13033076  0x6d58c0a8a97839a4b038ffe06f6198662da1979e   
1     13033312  0xa3fb45696ac592cf4187798b6b17316c5d93460d   
2     13033783  0xd5d15f5d0d9dbb

In [26]:
pool_abi = Abi(pool_path)
swap_sig = pool_abi.events["Swap"].signature()


query = f"""
with pc as ({pool_created_query})
select sw.block_num, sw.tx_hash,
       sw.dec['sender'] as sender,
       sw.dec['recipient'] as recipient,
       sw.dec['amount0'] as amount0,
       sw.dec['amount1'] as amount1,
       sw.dec['sqrtPriceX96'] as sqrt_price_x96,
       sw.dec['liquidity'] as liquidity,
       sw.dec['tick'] as tick
  from (select sl.block_num, sl.tx_hash,
               evm_decode(sl.topic1, sl.topic2, sl.topic3, sl.data, '{swap_sig}') as dec
          from pc, logs sl
         where sl.address = pc.pool
           and sl.block_num >= pc.block_num
           and sl.topic0 = evm_topic('{swap_sig}')) as sw
"""

# Note: This will download all the swaps.
# process_query(client, query)

# Explain
# print(client.get_sql("explain " + query, read_all=True).to_pandas()['plan'][1])


In [27]:

# This is used to benchmark the same work as the original query, but incurring less network trafic.
agg_query = f'''select count(*), max(a.amount0), max(a.liquidity), sum(a.amount0) from ({query}) a'''
process_query(client, agg_query)

select count(*), max(a.amount0), max(a.liquidity), sum(a.amount0) from (
with pc as (
    select pc.block_num,
       pc.dec['pool'] as pool,
       pc.dec['token0'] as token0,
       pc.dec['token1'] as token1,
       pc.dec['fee'] as fee,
       pc.dec['tickSpacing'] as tick_spacing
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'PoolCreated(address indexed token0,address indexed token1,uint24 indexed fee,int24 tickSpacing,address pool)') as dec
              from logs l
             where l.address = arrow_cast(x'1F98431c8aD98523631AE4a59f267346ea31F984', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('PoolCreated(address indexed token0,address indexed token1,uint24 indexed fee,int24 tickSpacing,address pool)')) pc
)
select sw.block_num, sw.tx_hash,
       sw.dec['sender'] as sender,
       sw.dec['recipient'] as recipient,
       sw.dec['amount0'] as amount0,
       sw.dec['amount1'] as amount1,
       sw.dec['sqr

In [28]:
graph_token_lock_manager = '0xFCf78AC094288D7200cfdB367A8CD07108dFa128'